In [1]:
##nesscity librares
#pandas used for data frame
import pandas as pd 
#numpy used for all math function
import numpy as np
#matplotlib sun fn of pyplot used for viusalize data in graphic
import matplotlib.pyplot as py
#For ML Purpose used libraises
#Cross_val to calculate the value of model
#kfold it used for how many times your test and train  data want to split. Each time it will give different of dataset based on kfold
#kfold Ex: a=[1,2,3,4,5,6] kfold=3 --> the 'a' dataset should be change follow upk[n-1]:k=3 a=[1,4,5,6,2,3] ,k=2 a=[6,5,4,3,2,1], k=1 a[3,2,1,6,4,5]
from sklearn.model_selection import cross_val_score,KFold
#the model used for calculated the accuracy_score of the mode . For this we should give a parameter of 'X[independent],Y[Depent]'
from sklearn.metrics import accuracy_score
#This model i used to create predict model of the project
#why i was used to project, the accuracy score is high compare to other model
from sklearn.svm import SVC,LinearSVC
#Term-Frequency Inverse Term Frequency[TFIDF],we can use counter vectorizer also but compare to other it was more efficient way to vectrozier of caterforical variable
from sklearn.feature_extraction.text import TfidfVectorizer



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [29]:
#with help pandas module we can read the json file 
train=pd.read_json('train.json')
test=pd.read_json("test.json")

#successfully read the json file


In [39]:
#Explority Data Analysis
#check is there any values are availabe in our data set
#cort_values fn used to asc our values form lower to higher
traisnull=train.isnull().sum().sort_values(ascending=True)
#Percent=n/N*100 like wise we used in here
trapercent=(train.isnull().sum()/train.isnull().count()*100).sort_values(ascending=True)
#shape is usually represent the number of records had it in Json file as a coloum,row wise
print('The Total row& coloumn of dataframe',train.shape)
#concat fn is available in pandas lib. It's used for merge with data. axis = 0 represent for Row value vice versa axis=1 represent of column 
both=pd.concat([traisnull,trapercent],axis=1,keys=['NULL','Percent'])
#understading purpose i used it print
print(both)

#here, we can see, we dont have that much numm values, if it we have to handle this missing values with help of Imputer fn.

The Total row& coloumn of dataframe (39774, 3)
             NULL  Percent
cuisine         0      0.0
id              0      0.0
ingredients     0      0.0


TypeError: tuple indices must be integers or slices, not tuple

In [40]:
#this is used to view the number of cuisine in this dataset. 
#whereever we are using groupby fn at time we did used sort_values fn , ought to give 'by' parameter.
group=train.groupby('cuisine').count().sort_values(by='id', ascending=False)
print(group)

                id  ingredients
cuisine                        
italian       7838         7838
mexican       6438         6438
southern_us   4320         4320
indian        3003         3003
chinese       2673         2673
french        2646         2646
cajun_creole  1546         1546
thai          1539         1539
japanese      1423         1423
greek         1175         1175
spanish        989          989
korean         830          830
vietnamese     825          825
moroccan       821          821
british        804          804
filipino       755          755
irish          667          667
jamaican       526          526
russian        489          489
brazilian      467          467


In [46]:
#the above used to view the data raw
#here it funtime
# Graphic view of data
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
#with help of groupby fn we came to know count of cuisine. so we have create 20 colour theme 
color_theme=dict(color=['rgba(221,160,221,1)','rgba(169,169,169,1)','rgba(255,160,122,1)','rgba(176,224,230,1)','rgba(169,169,169,1)','rgba(255,160,122,1)','rgba(176,224,230,1)',
                   'rgba(188,143,143,1)','rgba(221,160,221,1)','rgba(169,169,169,1)','rgba(255,160,122,1)','rgba(176,224,230,1)','rgba(189,183,107,1)','rgba(188,143,143,1)','rgba(221,160,221,1)','rgba(169,169,169,1)','rgba(255,160,122,1)','rgba(176,224,230,1)','rgba(169,169,169,1)','rgba(255,160,122,1)'])
#it count the particular cuisine of ingredients. Ex: 'Cuisine': british        value_count:804 
obj=train['cuisine'].value_counts()

#This is 2D of Graph, we need x and y co-oridinate. Orientation= h represent of 'horizontal' 
#marker= her e we used to our dict values of color_theme

template =go.Bar(y=obj.index[::-1],x=(obj)[::-1],orientation='h',marker=color_theme)

#layout is used for border of our graph. usually title, margin,fontsize values we assign with help of graph_objs module

layout=go.Layout(title='Grapics verision of cuisine',xaxis=dict(title='count of ingridents',tickfont=dict(size=14,)),
                yaxis=dict(title='List of orgin',titlefont=dict(size=16),tickfont=dict(size=14)),margin=dict(l=200,))
data=[template]
figure=go.Figure(data=data,layout=layout)
iplot(figure,filename='basic-bar')

In [49]:
# used to read  json without help of pandas
#defentiely it will make curisoity 
# call the module of Json in  python 
import json
# define the fn name
def read_json(path):
    #return my caller open the specified path
    return json.load(open(path))
#call the  fn and create the object too
train=read_json('train.json')
test=read_json('test.json')
#have data with out dataframe 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
#separate ingredients from the test and train with the help of iteration 
def create_text(data):
    #might be non-numberic data data value lower and uppder case. for normalize purpose used to lower()
    actualdata=(''.join(doc['ingredients']).lower() for doc in data)
    return actualdata
utrain=create_text(train)
utest=create_text(test)
#both utrain(u represent of update),utest having an ingredients values
#Independent variable ready to proceed into model


In [5]:
#Target(depent variable) from the test and train with the help of iteration 
target= [doc['cuisine']for doc in train]
#crate an object for Tfidfvectorizer. We should to vectorizer because we have data of text values.Machine can understand of numberice varibale
#which is can used to text to numberic value[Tfidf]
tfidf=TfidfVectorizer(binary=True)

#X[indepent],y[depent] are serve to ready

In [6]:
#call the tf-idf which used to change out string values into machine understand values of numeric
def final(obj,identify):
    if identify=='train':
        #fit_transform and transform is used for nomalize your data in a particular range
        #we have formual for z score=x-(mu)/sigma [normalize]
        #the above formula give your data on a particular range
        #fit(): is used to calculate of parameterof  mu and sigma value of your train data and save it in a internal objects
        #transform: is used for the above calculated parameter we can transform for an particular dataset
        #fit_transform: calculate the parameters and trasformer to the train dataset
        Pred=tfidf.fit_transform(obj)
        
    else:
        #transform(): is used for the calculate the paramters transformed in to test dataset.
        Pred=tfidf.transform(obj)
    return Pred
Train=final(utrain,'train')
Test=final(utest,'test')

#All  str values converted into numberic values
#tf-idf: ex: n=['tea','mango','apple','orange','tea']
#tf-idf formual: TF= number of term present in 'n'/Total number of n --> tea[TF]=2/5=0.4
                #IDF= log(Total number of n/Number of term present in 'n') -->tea[IDF]= log(5/2)= log(2.5)=0.40
                #so Tea if- idf value is =TF*IDF=0.4*0.40=0.16


In [7]:
#model Preparation
from sklearn.preprocessing import LabelEncoder
#using label encoder give a weightage of target variable
#create an object of LabelEncoder

lb=LabelEncoder()
Target=lb.fit_transform(target)


#Target variable ready

In [14]:
#used in LinearSVC 
#SVC kernal of linear is used to fit to the date provide to 'bestfit' into your model,that divide categorizes,after getting the hyperplane.
from sklearn import model_selection
from sklearn.model_selection import KFold
#create an object for linearsvc
lc=LinearSVC()
perfect=lc.fit(Train,Target)
#the above steps used to check my accuracy value
#accuracy value: how much of my predict data similar to the actual data
#here my kfold value id 10 is will be reduce like k(n-1) that's mean k(10-1)
kflod= model_selection.KFold(n_splits=10,random_state=7)
Acc_results = model_selection.cross_val_score(LinearSVC(),Train,Target,cv=kflod,scoring='accuracy')
print('The Linear SVC Result is ',Acc_results.mean(),Acc_results.std())





The Linear SVC Result is  0.6254084603867917 0.004007452261871983


In [19]:
#Our ML model used for Test data
#in Test data we have, calculate the paramters transformed

y_test=lc.predict(Test)
y_pred=lb.inverse_transform(y_test)
#test data have the whole dataset, in that we are extracting the id column
Test_Pre=[doc['id'] for doc in test]
Res=pd.DataFrame({'id':Test_Pre,'cuisine':y_pred},columns=['id','cuisine'])


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [56]:
#Here is the climax of the test data prediction and result of accuracy
Acc_results1 = model_selection.cross_val_score(LinearSVC(),Test,y_pred,cv=kflod,scoring='accuracy')
print('The Linear SVC Result of Prediction ',Acc_results1.mean(),Acc_results1.std())

         id       cuisine
0     18009      japanese
1     28583   southern_us
2     41580       italian
3     29752  cajun_creole
4     35687       italian
5     38527        french
6     19666       italian
7     41217       chinese
8     28753       mexican
9     22659       british
10    21749       italian
11    44967         greek
12    42969        indian
13    44883       italian
14    20827   southern_us
15    23196       mexican
16    35387   southern_us
17    33780        french
18    19001       mexican
19    16526       mexican
20    42455      japanese
21    47453  cajun_creole
22    42478   southern_us
23    11885    vietnamese
24    16585       italian
25    29639   southern_us
26    26245          thai
27    38516       chinese
28    47520       italian
29    26212   southern_us
...     ...           ...
9914  49157    vietnamese
9915  40847       italian
9916  14084   southern_us
9917   6802       italian
9918  22381       mexican
9919  21016       mexican
9920  29024 

The Linear SVC Result of Prediction  0.7228474363770563 0.012963908565603843
